![Team USA Football](images/Usa_football_body_logo.png)

Data Dictionary here https://nflreadr.nflverse.com/articles/dictionary_player_stats.html

In [1]:
import pandas as pd
import numpy as np
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout

ill save my data as some new dataframes and see what i have.

In [2]:
stats_df = pd.read_csv('data/player_stats.csv')
qbr_df = pd.read_csv('data/qbr_season_level.csv') 

ill start with stats_df as that will require more work

In [3]:
stats_df.head()

,player_id,player_name,player_display_name,position,position_group,headshot_url,recent_team,season,week,season_type,...,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr
0,00-0000003,NaN,Abdul-Karim al-Jabbar,RB,RB,NaN,MIA,1999,1,REG,...,0,0.292378,0,0.0,0.052632,NaN,NaN,0,12.7,13.7
1,00-0000003,NaN,Abdul-Karim al-Jabbar,RB,RB,NaN,MIA,1999,2,REG,...,1,0.377009,0,0.0,0.117647,NaN,NaN,0,5.1,8.1
2,00-0000003,NaN,Abdul-Karim al-Jabbar,RB,RB,NaN,MIA,1999,4,REG,...,0,-0.699578,0,NaN,0.023810,NaN,NaN,0,0.2,0.2
3,00-0000003,NaN,Abdul-Karim al-Jabbar,RB,RB,NaN,CLE,1999,7,REG,...,0,-0.228454,0,0.0,0.050000,NaN,NaN,0,3.5,5.5
4,00-0000003,NaN,Abdul-Karim al-Jabbar,RB,RB,NaN,CLE,1999,8,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,3.9,3.9


In [4]:
# lets check up on Aaron Rodgers
stats_df[stats_df['player_display_name'] == 'Aaron Rodgers'].head()

,player_id,player_name,player_display_name,position,position_group,headshot_url,recent_team,season,week,season_type,...,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr
34491,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/private/f_aut...,GB,2005,5,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,0.00,0.00
34492,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/private/f_aut...,GB,2005,15,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,-2.60,-2.60
34493,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/private/f_aut...,GB,2005,17,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,-0.10,-0.10
38709,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/private/f_aut...,GB,2006,4,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,0.56,0.56
38710,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/private/f_aut...,GB,2006,11,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,0.38,0.38


In [5]:
stats_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129739 entries, 0 to 129738
Data columns (total 53 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   player_id                    129739 non-null  object 
 1   player_name                  61504 non-null   object 
 2   player_display_name          129736 non-null  object 
 3   position                     129667 non-null  object 
 4   position_group               129667 non-null  object 
 5   headshot_url                 69803 non-null   object 
 6   recent_team                  129739 non-null  object 
 7   season                       129739 non-null  int64  
 8   week                         129739 non-null  int64  
 9   season_type                  129739 non-null  object 
 10  opponent_team                129739 non-null  object 
 11  completions                  129739 non-null  int64  
 12  attempts                     129739 non-null  int64  
 13 

In [6]:
# Use 1 to show all info in cell below then comment it out and use 2 to go back to default view I can delete this later

#1
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

#2
#pd.reset_option('display.max_rows')
#pd.reset_option('display.max_columns')
#pd.reset_option('display.max_colwidth')


In [7]:
stats_df['position'].value_counts()

WR     49363
RB     33350
TE     24147
QB     16086
FB      5219
P        459
CB       148
G        128
DB       123
T         94
FS        93
SS        77
LS        56
OLB       56
K         46
LB        45
ILB       32
C         28
DE        28
DT        20
OT        17
DL        15
HB        13
NT        10
MLB        8
OG         3
OL         2
S          1
Name: position, dtype: int64

ive only got offensive related stats going back to 1999. I wont be needing those older players in a team for today

In [8]:
#lets look at the last ten years of data 
stats_df = stats_df[stats_df['season'].isin(range(2014, 2024))]

In [9]:
#lets only look at the regular season so we can see every player in action. this is about players not teams
stats_df = stats_df[stats_df['season_type'] == 'REG']


again these are offensive stats but they do give me a few key pieces of info that i can credit the oppenent team with. the opponent of an offense being.....a defense!!

# Creating Defense 

In [10]:
# Define a function to handle grouping, summing, renaming, and filling NaNs
def process_stat(stat_df, stat_name, new_name=None):
    if new_name is None:
        new_name = stat_name
    grouped_data = stat_df.groupby(['opponent_team', 'season'])[stat_name].sum().reset_index()
    grouped_data.rename(columns={'opponent_team': 'defense', stat_name: new_name}, inplace=True)
    grouped_data.fillna(0, inplace=True)
    return grouped_data

# List of statistics to process
stats_to_process = [
    ('interceptions', 'interceptions_total'),
    ('sacks', 'sacks'),
    ('sack_yards', 'sack_yards'),
    ('receiving_yards', 'receiving_yards'),
    ('rushing_yards', 'rushing_yards'),
    ('rushing_tds', 'rushing_tds'),
    ('receiving_tds', 'receiving_tds'),
    ('sack_fumbles', 'sack_fumbles'),
    ('rushing_fumbles', 'rushing_fumbles'),
    ('receiving_fumbles', 'receiving_fumbles'),
    ('sack_fumbles_lost', 'sack_fumbles_recovered'),
    ('rushing_fumbles_lost', 'rushing_fumbles_recovered'),
    ('receiving_fumbles_lost', 'receiving_fumbles_recovered')
]

# Process each stat and store in a list of DataFrames
processed_stats = [process_stat(stats_df, stat, new_name) for stat, new_name in stats_to_process]

# Merge all DataFrames on 'defense' and 'season'
defense_df = processed_stats[0]
for df in processed_stats[1:]:
    defense_df = pd.merge(defense_df, df, on=['defense', 'season'], how='outer')

# Display the resulting DataFrame
defense_df.head()

,defense,season,interceptions_total,sacks,sack_yards,receiving_yards,rushing_yards,rushing_tds,receiving_tds,sack_fumbles,rushing_fumbles,receiving_fumbles,sack_fumbles_recovered,rushing_fumbles_recovered,receiving_fumbles_recovered
0,ARI,2014,18,35,244,4396,1739,9,22,2,4,4,1,1,4
1,ARI,2015,19,36,270,3957,1460,9,24,8,12,4,5,5,2
2,ARI,2016,14,48,374,3739,1518,16,21,12,11,2,6,4,1
3,ARI,2017,15,37,255,3796,1434,12,24,7,7,3,2,1,2
4,ARI,2018,7,49,333,3595,2479,25,21,7,3,5,6,1,2


In [11]:
defense_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 320 entries, 0 to 319
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   defense                      320 non-null    object
 1   season                       320 non-null    int64 
 2   interceptions_total          320 non-null    int64 
 3   sacks                        320 non-null    int64 
 4   sack_yards                   320 non-null    int64 
 5   receiving_yards              320 non-null    int64 
 6   rushing_yards                320 non-null    int64 
 7   rushing_tds                  320 non-null    int64 
 8   receiving_tds                320 non-null    int64 
 9   sack_fumbles                 320 non-null    int64 
 10  rushing_fumbles              320 non-null    int64 
 11  receiving_fumbles            320 non-null    int64 
 12  sack_fumbles_recovered       320 non-null    int64 
 13  rushing_fumbles_recovered    320 no

just need to create some ratios and totals for defense and that should be ready 

In [12]:
# Create forced_fumbles column
defense_df['forced_fumbles'] = (defense_df['sack_fumbles'] +
                                defense_df['rushing_fumbles'] +
                                defense_df['receiving_fumbles']
                               )
# Create fumbles_recovered column
defense_df['fumbles_recovered'] = (defense_df['sack_fumbles_recovered'] +
                                   defense_df['rushing_fumbles_recovered'] +
                                   defense_df['receiving_fumbles_recovered']
                                  )

# Calculate the percentage of forced fumbles that were fumbles recovered
defense_df['percentage_fumbles_recovered'] = (defense_df['fumbles_recovered'] / defense_df['forced_fumbles'] * 100).fillna(0)

# Create tds_against column
defense_df['tds_against'] = (defense_df['rushing_tds'] +
                             defense_df['receiving_tds']
                            )

#Create yards_against column
defense_df['yards_against'] = (defense_df['rushing_yards'] +
                             defense_df['receiving_yards'] -
                             defense_df['sack_yards']
                            )

# Display the updated DataFrame
defense_df.head()

,defense,season,interceptions_total,sacks,sack_yards,receiving_yards,rushing_yards,rushing_tds,receiving_tds,sack_fumbles,rushing_fumbles,receiving_fumbles,sack_fumbles_recovered,rushing_fumbles_recovered,receiving_fumbles_recovered,forced_fumbles,fumbles_recovered,percentage_fumbles_recovered,tds_against,yards_against
0,ARI,2014,18,35,244,4396,1739,9,22,2,4,4,1,1,4,10,6,60.000000,31,5891
1,ARI,2015,19,36,270,3957,1460,9,24,8,12,4,5,5,2,24,12,50.000000,33,5147
2,ARI,2016,14,48,374,3739,1518,16,21,12,11,2,6,4,1,25,11,44.000000,37,4883
3,ARI,2017,15,37,255,3796,1434,12,24,7,7,3,2,1,2,17,5,29.411765,36,4975
4,ARI,2018,7,49,333,3595,2479,25,21,7,3,5,6,1,2,15,9,60.000000,46,5741


In [13]:
defense_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 320 entries, 0 to 319
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   defense                       320 non-null    object 
 1   season                        320 non-null    int64  
 2   interceptions_total           320 non-null    int64  
 3   sacks                         320 non-null    int64  
 4   sack_yards                    320 non-null    int64  
 5   receiving_yards               320 non-null    int64  
 6   rushing_yards                 320 non-null    int64  
 7   rushing_tds                   320 non-null    int64  
 8   receiving_tds                 320 non-null    int64  
 9   sack_fumbles                  320 non-null    int64  
 10  rushing_fumbles               320 non-null    int64  
 11  receiving_fumbles             320 non-null    int64  
 12  sack_fumbles_recovered        320 non-null    int64  
 13  rushi

In [14]:
defense_df.head()

,defense,season,interceptions_total,sacks,sack_yards,receiving_yards,rushing_yards,rushing_tds,receiving_tds,sack_fumbles,rushing_fumbles,receiving_fumbles,sack_fumbles_recovered,rushing_fumbles_recovered,receiving_fumbles_recovered,forced_fumbles,fumbles_recovered,percentage_fumbles_recovered,tds_against,yards_against
0,ARI,2014,18,35,244,4396,1739,9,22,2,4,4,1,1,4,10,6,60.000000,31,5891
1,ARI,2015,19,36,270,3957,1460,9,24,8,12,4,5,5,2,24,12,50.000000,33,5147
2,ARI,2016,14,48,374,3739,1518,16,21,12,11,2,6,4,1,25,11,44.000000,37,4883
3,ARI,2017,15,37,255,3796,1434,12,24,7,7,3,2,1,2,17,5,29.411765,36,4975
4,ARI,2018,7,49,333,3595,2479,25,21,7,3,5,6,1,2,15,9,60.000000,46,5741


these stats dont tell the full story, i dont know who really won the game as i have no clue if a defense scored, i only know that they were scored on so lets use that and only that for now 

i dont think i use any statistical models for the defense. ill make visualizations here later 

# The Offense

In [15]:
#now lets move back to the offense and seperate by position for our 3 analyses
# Filter for quarterbacks
qbs_df = stats_df[stats_df['position'] == 'QB']

# Filter for rushers (RBs and FBs)
rushers_df = stats_df[stats_df['position'].isin(['RB', 'FB', 'HB'])]

# Filter for receivers (WRs and TEs)
receivers_df = stats_df[stats_df['position'].isin(['WR', 'TE'])]

In [16]:
# Create efficiency metrics
qbs_df.loc[:, 'completion_percentage'] = qbs_df['completions'] / qbs_df['attempts']
qbs_df.loc[:, 'yards_per_attempt'] = qbs_df['passing_yards'] / qbs_df['attempts']
rushers_df.loc[:, 'yards_per_carry'] = rushers_df['rushing_yards'] / rushers_df['carries']
receivers_df.loc[:, 'yards_per_reception'] = receivers_df['receiving_yards'] / receivers_df['receptions']

# Create touchdown ratios
qbs_df.loc[:, 'pass_tds_per_attempt'] = qbs_df['passing_tds'] / qbs_df['attempts']
rushers_df.loc[:, 'tds_per_carry'] = rushers_df['rushing_tds'] / rushers_df['carries']
receivers_df.loc[:, 'tds_per_reception'] = receivers_df['receiving_tds'] / receivers_df['receptions']

# Calculate season stats for each player over the season
qbs_df.loc[:, 'season_passing_yards'] = qbs_df.groupby(['player_id', 'season'])['passing_yards'].transform('sum')
rushers_df.loc[:, 'season_rushing_yards'] = rushers_df.groupby(['player_id', 'season'])['rushing_yards'].transform('sum')
receivers_df.loc[:, 'season_receiving_yards'] = receivers_df.groupby(['player_id', 'season'])['receiving_yards'].transform('sum')
qbs_df.loc[:, 'season_passing_tds'] = qbs_df.groupby(['player_id', 'season'])['passing_tds'].transform('sum')
rushers_df.loc[:, 'season_rushing_tds'] = rushers_df.groupby(['player_id', 'season'])['rushing_tds'].transform('sum')
receivers_df.loc[:, 'season_receiving_tds'] = receivers_df.groupby(['player_id', 'season'])['receiving_tds'].transform('sum')
qbs_df.loc[:, 'season_pass_attempts'] = qbs_df.groupby(['player_id', 'season'])['attempts'].transform('sum')

# Calculate season completion percentage
qbs_df.loc[:, 'season_completion_percentage'] = qbs_df.groupby(['player_id', 'season'])['completion_percentage'].transform('mean')
qbs_df.loc[:, 'season_yards_per_attempt'] = qbs_df.groupby(['player_id', 'season'])['yards_per_attempt'].transform('mean')
qbs_df.loc[:, 'season_pass_tds_per_attempt'] = qbs_df.groupby(['player_id', 'season'])['pass_tds_per_attempt'].transform('mean')

c:\Users\Trugrit\anaconda3\envs\learn-env\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
c:\Users\Trugrit\anaconda3\envs\learn-env\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


## Drop unnesscessary columns from each df

In [17]:
qbs_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6211 entries, 75507 to 129722
Data columns (total 62 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   player_id                     6211 non-null   object 
 1   player_name                   5993 non-null   object 
 2   player_display_name           6211 non-null   object 
 3   position                      6211 non-null   object 
 4   position_group                6211 non-null   object 
 5   headshot_url                  6154 non-null   object 
 6   recent_team                   6211 non-null   object 
 7   season                        6211 non-null   int64  
 8   week                          6211 non-null   int64  
 9   season_type                   6211 non-null   object 
 10  opponent_team                 6211 non-null   object 
 11  completions                   6211 non-null   int64  
 12  attempts                      6211 non-null   int64  
 1

In [18]:
qbs_df = qbs_df.drop(columns= ['player_id',
                               'player_name',
                               'position', 
                               'position_group',
                               'headshot_url',
                               'season_type',   
                               'passing_epa',
                               'pacr',
                               'dakota',
                               'attempts',
                               'completions',
                               'passing_yards',
                               'passing_tds',
                               'interceptions',
                               'sacks',
                               'sack_yards',
                               'sack_fumbles',
                               'sack_fumbles_lost',
                               'passing_air_yards',
                               'passing_yards_after_catch',
                               'passing_first_downs',
                               'passing_epa',
                               'passing_2pt_conversions',
                               'carries',
                               'rushing_yards',
                               'rushing_tds',
                               'rushing_fumbles',
                               'rushing_fumbles_lost',
                               'rushing_first_downs',
                               'rushing_epa',
                               'rushing_2pt_conversions',
                               'receptions',
                               'targets',
                               'receiving_yards',
                               'receiving_tds',
                               'receiving_fumbles',
                               'receiving_fumbles_lost',
                               'receiving_air_yards',
                               'receiving_yards_after_catch',
                               'receiving_first_downs',
                               'receiving_epa',
                               'receiving_2pt_conversions',
                               'racr',
                               'target_share',
                               'air_yards_share',
                               'wopr',
                               'special_teams_tds',
                               'fantasy_points',
                               'fantasy_points_ppr'
                              ]).reset_index(drop=True)
qbs_df.head()

,player_display_name,recent_team,season,week,opponent_team,completion_percentage,yards_per_attempt,pass_tds_per_attempt,season_passing_yards,season_passing_tds,season_pass_attempts,season_completion_percentage,season_yards_per_attempt,season_pass_tds_per_attempt
0,Matt Hasselbeck,IND,2014,3,JAX,0.500000,5.000000,0.000000,301,2,44,0.632832,6.385965,0.033417
1,Matt Hasselbeck,IND,2014,4,TEN,NaN,NaN,NaN,301,2,44,0.632832,6.385965,0.033417
2,Matt Hasselbeck,IND,2014,16,DAL,0.714286,6.000000,0.047619,301,2,44,0.632832,6.385965,0.033417
3,Matt Hasselbeck,IND,2014,17,TEN,0.684211,8.157895,0.052632,301,2,44,0.632832,6.385965,0.033417
4,Peyton Manning,DEN,2014,1,IND,0.611111,7.472222,0.083333,4727,39,597,0.673296,8.169275,0.068182


In [19]:
# Get the index of the maximum week for each player
max_week_idx = qbs_df.groupby(['player_display_name', 'season'])['week'].idxmax()

# Select the rows corresponding to these indices
qbs_df = qbs_df.loc[max_week_idx].reset_index(drop=True)
qbs_df = qbs_df.drop(columns= 'week')
qbs_df.head()

,player_display_name,recent_team,season,opponent_team,completion_percentage,yards_per_attempt,pass_tds_per_attempt,season_passing_yards,season_passing_tds,season_pass_attempts,season_completion_percentage,season_yards_per_attempt,season_pass_tds_per_attempt
0,AJ McCarron,CIN,2015,BAL,0.629630,5.925926,0.074074,854,6,119,0.721109,6.477734,0.035461
1,AJ McCarron,CIN,2017,MIN,0.500000,3.166667,0.000000,66,0,14,0.500000,4.520833,0.000000
2,AJ McCarron,LV,2018,BAL,NaN,NaN,NaN,8,0,3,0.333333,2.666667,0.000000
3,AJ McCarron,HOU,2019,TEN,0.583333,6.250000,0.000000,225,0,37,0.291667,3.125000,0.000000
4,AJ McCarron,HOU,2020,TEN,1.000000,20.000000,0.000000,20,0,1,1.000000,20.000000,0.000000


In [20]:
qbs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 769 entries, 0 to 768
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   player_display_name           769 non-null    object 
 1   recent_team                   769 non-null    object 
 2   season                        769 non-null    int64  
 3   opponent_team                 769 non-null    object 
 4   completion_percentage         693 non-null    float64
 5   yards_per_attempt             693 non-null    float64
 6   pass_tds_per_attempt          693 non-null    float64
 7   season_passing_yards          769 non-null    int64  
 8   season_passing_tds            769 non-null    int64  
 9   season_pass_attempts          769 non-null    int64  
 10  season_completion_percentage  737 non-null    float64
 11  season_yards_per_attempt      737 non-null    float64
 12  season_pass_tds_per_attempt   737 non-null    float64
dtypes: fl

clean nan

In [21]:
qbs_df = qbs_df.dropna(subset=['completion_percentage'])

In [22]:
qbs_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 693 entries, 0 to 768
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   player_display_name           693 non-null    object 
 1   recent_team                   693 non-null    object 
 2   season                        693 non-null    int64  
 3   opponent_team                 693 non-null    object 
 4   completion_percentage         693 non-null    float64
 5   yards_per_attempt             693 non-null    float64
 6   pass_tds_per_attempt          693 non-null    float64
 7   season_passing_yards          693 non-null    int64  
 8   season_passing_tds            693 non-null    int64  
 9   season_pass_attempts          693 non-null    int64  
 10  season_completion_percentage  693 non-null    float64
 11  season_yards_per_attempt      693 non-null    float64
 12  season_pass_tds_per_attempt   693 non-null    float64
dtypes: fl

In [23]:
rushers_df = rushers_df.drop(columns= ['player_id',
                                       'player_name',
                                       'position', 
                                       'position_group',
                                       'headshot_url',
                                       'season_type',
                                       'passing_epa',
                                       'pacr',
                                       'dakota',
                                       'completions',
                                       'attempts',
                                       'passing_yards',
                                       'passing_tds',
                                       'interceptions',
                                       'sacks',
                                       'sack_yards',
                                       'sack_fumbles',
                                       'sack_fumbles_lost',
                                       'passing_air_yards',
                                       'passing_yards_after_catch',
                                       'passing_first_downs',
                                       'passing_epa',
                                       'passing_2pt_conversions',
                                       'rushing_epa',
                                       'receptions',
                                       'targets',
                                       'receiving_yards',
                                       'receiving_tds',
                                       'receiving_fumbles',
                                       'receiving_fumbles_lost',
                                       'receiving_air_yards',
                                       'receiving_yards_after_catch',
                                       'receiving_first_downs',
                                       'receiving_epa',
                                       'receiving_2pt_conversions',
                                       'racr',
                                       'target_share',
                                       'air_yards_share',
                                       'wopr',
                                       'special_teams_tds',
                                       'fantasy_points',
                                       'fantasy_points_ppr'
                                      ]).reset_index(drop=True)

rushers_df.head()

,player_display_name,recent_team,season,week,opponent_team,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_2pt_conversions,yards_per_carry,tds_per_carry,season_rushing_yards,season_rushing_tds
0,Steven Jackson,ATL,2014,1,NO,12,52,0,0,0,2,0,4.333333,0.000000,707,6
1,Steven Jackson,ATL,2014,2,CIN,11,46,0,0,0,2,0,4.181818,0.000000,707,6
2,Steven Jackson,ATL,2014,3,TB,14,54,1,0,0,5,0,3.857143,0.071429,707,6
3,Steven Jackson,ATL,2014,4,MIN,13,49,0,0,0,3,0,3.769231,0.000000,707,6
4,Steven Jackson,ATL,2014,5,NYG,13,37,1,0,0,3,0,2.846154,0.076923,707,6


In [24]:
rushers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14451 entries, 0 to 14450
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   player_display_name      14451 non-null  object 
 1   recent_team              14451 non-null  object 
 2   season                   14451 non-null  int64  
 3   week                     14451 non-null  int64  
 4   opponent_team            14451 non-null  object 
 5   carries                  14451 non-null  int64  
 6   rushing_yards            14451 non-null  int64  
 7   rushing_tds              14451 non-null  int64  
 8   rushing_fumbles          14451 non-null  int64  
 9   rushing_fumbles_lost     14451 non-null  int64  
 10  rushing_first_downs      14451 non-null  int64  
 11  rushing_2pt_conversions  14451 non-null  int64  
 12  yards_per_carry          12959 non-null  float64
 13  tds_per_carry            12959 non-null  float64
 14  season_rushing_yards  

drop nan

In [25]:
receivers_df = receivers_df.drop(columns= ['player_id',
                                           'player_name',
                                           'position', 
                                           'position_group',
                                           'headshot_url',
                                           'season_type',
                                           'passing_epa',
                                           'pacr',
                                           'dakota',
                                           'completions',
                                           'attempts',
                                           'passing_yards',
                                           'passing_tds',
                                           'interceptions',
                                           'sacks',
                                           'sack_yards',
                                           'sack_fumbles',
                                           'sack_fumbles_lost',
                                           'passing_air_yards',
                                           'passing_yards_after_catch',
                                           'passing_first_downs',
                                           'passing_epa',
                                           'passing_2pt_conversions',
                                           'carries',
                                           'rushing_yards',
                                           'rushing_tds',
                                           'rushing_fumbles',
                                           'rushing_fumbles_lost',
                                           'rushing_first_downs',
                                           'rushing_epa',
                                           'rushing_2pt_conversions',
                                           'receiving_epa',
                                           'racr',
                                           'target_share',
                                           'air_yards_share',
                                           'wopr',
                                           'special_teams_tds',
                                           'fantasy_points',
                                           'fantasy_points_ppr'
                                          ]).reset_index(drop=True)

receivers_df.head()

,player_display_name,recent_team,season,week,opponent_team,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_2pt_conversions,yards_per_reception,tds_per_reception,season_receiving_yards,season_receiving_tds
0,Steve Smith,BAL,2014,1,CIN,7,15,118,1,0,0,145,51,3,0,16.857143,0.142857,1065,6
1,Steve Smith,BAL,2014,2,PIT,6,10,71,0,0,0,87,45,4,0,11.833333,0.000000,1065,6
2,Steve Smith,BAL,2014,3,CLE,5,7,101,0,0,0,95,25,3,0,20.200000,0.000000,1065,6
3,Steve Smith,BAL,2014,4,CAR,7,10,139,2,0,0,102,69,6,0,19.857143,0.285714,1065,6
4,Steve Smith,BAL,2014,5,IND,5,7,34,0,1,1,42,7,1,0,6.800000,0.000000,1065,6


In [26]:
receivers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30802 entries, 0 to 30801
Data columns (total 19 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   player_display_name          30802 non-null  object 
 1   recent_team                  30802 non-null  object 
 2   season                       30802 non-null  int64  
 3   week                         30802 non-null  int64  
 4   opponent_team                30802 non-null  object 
 5   receptions                   30802 non-null  int64  
 6   targets                      30802 non-null  int64  
 7   receiving_yards              30802 non-null  int64  
 8   receiving_tds                30802 non-null  int64  
 9   receiving_fumbles            30802 non-null  int64  
 10  receiving_fumbles_lost       30802 non-null  int64  
 11  receiving_air_yards          30802 non-null  int64  
 12  receiving_yards_after_catch  30802 non-null  int64  
 13  receiving_first_

### Feature Engineering

In [27]:
# checking out well known *cough JETS cough* players with essential features
qbs_df[qbs_df['player_display_name'] == 'Aaron Rodgers'].head()

#receivers_df[receivers_df['player_display_name'] == 'Garrett Wilson'].head()

#rushers_df[rushers_df['player_display_name'] == 'Breece Hall'].head()


,player_display_name,recent_team,season,opponent_team,completion_percentage,yards_per_attempt,pass_tds_per_attempt,season_passing_yards,season_passing_tds,season_pass_attempts,season_completion_percentage,season_yards_per_attempt,season_pass_tds_per_attempt
6,Aaron Rodgers,GB,2014,DET,0.772727,10.272727,0.090909,4381,38,520,0.669568,8.641434,0.082370
7,Aaron Rodgers,GB,2015,MIN,0.636364,6.613636,0.022727,3821,31,572,0.615191,6.733264,0.055985
8,Aaron Rodgers,GB,2016,DET,0.692308,7.692308,0.102564,4428,40,610,0.659904,7.421580,0.069920
9,Aaron Rodgers,GB,2017,CAR,0.577778,6.444444,0.066667,1675,16,238,0.631227,6.738127,0.065600
10,Aaron Rodgers,GB,2018,DET,0.600000,5.200000,0.000000,4442,25,597,0.624470,7.423917,0.041747


create targets for each df

## QBR

just need to pull out QBR by season to go along with other QB stats

In [28]:
qbr_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1363 entries, 0 to 1362
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   season         1363 non-null   int64  
 1   season_type    1363 non-null   object 
 2   game_week      1363 non-null   object 
 3   team_abb       1363 non-null   object 
 4   player_id      1363 non-null   int64  
 5   name_short     1363 non-null   object 
 6   rank           775 non-null    float64
 7   qbr_total      1363 non-null   float64
 8   pts_added      1363 non-null   float64
 9   qb_plays       1363 non-null   int64  
 10  epa_total      1363 non-null   float64
 11  pass           1363 non-null   float64
 12  run            1363 non-null   float64
 13  exp_sack       1363 non-null   int64  
 14  penalty        1363 non-null   float64
 15  qbr_raw        1363 non-null   float64
 16  sack           1363 non-null   float64
 17  name_first     1363 non-null   object 
 18  name_las

In [29]:
qbr_df = qbr_df[qbr_df['season_type'] == 'Regular']

In [30]:
# Define the columns to keep
columns_to_keep = ['season', 'qbr_total', 'name_display']

# Drop columns from qbr_df except the specified ones
qbr_df = qbr_df[columns_to_keep]

# Display the modified qbr_df
qbr_df.head()

,season,qbr_total,name_display
0,2006,86.4,Peyton Manning
1,2006,68.6,Tom Brady
2,2006,67.4,Philip Rivers
3,2006,67.1,Carson Palmer
4,2006,66.7,Drew Brees


In [31]:
# Define the mapping between columns in qbs_df and qbr_df
column_mapping = {'player_display_name': 'name_display', 'season': 'season'}

# Merge DataFrames using the mapped columns with an inner join
qbs_df = pd.merge(qbs_df, qbr_df, left_on=['player_display_name', 'season'], 
                  right_on=['name_display', 'season'], how='inner')

# Drop the redundant 'name_display' column
qbs_df.drop(columns=['name_display'], inplace=True)

# Display the merged DataFrame
qbs_df.head()

,player_display_name,recent_team,season,opponent_team,completion_percentage,yards_per_attempt,pass_tds_per_attempt,season_passing_yards,season_passing_tds,season_pass_attempts,season_completion_percentage,season_yards_per_attempt,season_pass_tds_per_attempt,qbr_total
0,AJ McCarron,CIN,2015,BAL,0.629630,5.925926,0.074074,854,6,119,0.721109,6.477734,0.035461,54.8
1,Aaron Rodgers,GB,2014,DET,0.772727,10.272727,0.090909,4381,38,520,0.669568,8.641434,0.082370,77.8
2,Aaron Rodgers,GB,2015,MIN,0.636364,6.613636,0.022727,3821,31,572,0.615191,6.733264,0.055985,60.0
3,Aaron Rodgers,GB,2016,DET,0.692308,7.692308,0.102564,4428,40,610,0.659904,7.421580,0.069920,72.4
4,Aaron Rodgers,GB,2017,CAR,0.577778,6.444444,0.066667,1675,16,238,0.631227,6.738127,0.065600,66.6


a QB needs to post at least 14 passes attempts per game to get a QBR. therefore any QB without one is irrelevant

In [32]:
qbs_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 405 entries, 0 to 404
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   player_display_name           405 non-null    object 
 1   recent_team                   405 non-null    object 
 2   season                        405 non-null    int64  
 3   opponent_team                 405 non-null    object 
 4   completion_percentage         405 non-null    float64
 5   yards_per_attempt             405 non-null    float64
 6   pass_tds_per_attempt          405 non-null    float64
 7   season_passing_yards          405 non-null    int64  
 8   season_passing_tds            405 non-null    int64  
 9   season_pass_attempts          405 non-null    int64  
 10  season_completion_percentage  405 non-null    float64
 11  season_yards_per_attempt      405 non-null    float64
 12  season_pass_tds_per_attempt   405 non-null    float64
 13  qbr_t

# Final Drop 

In [33]:
top_five_qbr = qbs_df.nlargest(5, 'qbr_total')
print(top_five_qbr)

    player_display_name recent_team  season opponent_team  \
158     Jacoby Brissett         IND    2018           MIA   
365       Sean Clifford          GB    2023           MIN   
348         Ryan Nassib         NYG    2015           MIN   
88         Chase Daniel         PHI    2016           NYG   
388           Tony Romo         DAL    2016           PHI   

     completion_percentage  yards_per_attempt  pass_tds_per_attempt  \
158                   1.00               4.00                  0.00   
365                   1.00              37.00                  0.00   
348                   1.00              13.60                  0.20   
88                    1.00              16.00                  0.00   
388                   0.75               7.25                  0.25   

     season_passing_yards  season_passing_tds  season_pass_attempts  \
158                     2                   0                     4   
365                    37                   0                   

In [34]:
qbs_df = qbs_df[qbs_df['season_pass_attempts'] >= 100]
qbs_df.drop(columns=['recent_team', 'opponent_team', 'completion_percentage', 'yards_per_attempt', 'pass_tds_per_attempt'], inplace=True)

In [35]:
# One-hot encode categorical features
qbs_df = pd.get_dummies(qbs_df, columns=['player_display_name'], drop_first=True)

# Separate features and target variable
X = qbs_df.drop(columns=['qbr_total'])
y = qbs_df['qbr_total']

# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Dummy

In [36]:
# Initialize the dummy regressor
dummy_regressor = DummyRegressor(strategy='mean')

# Train the dummy regressor
dummy_regressor.fit(X_train, y_train)

# Evaluate the dummy regressor
dummy_predictions = dummy_regressor.predict(X_test)
dummy_mae = mean_absolute_error(y_test, dummy_predictions)
print(f'Dummy Regressor MAE: {dummy_mae}')

Dummy Regressor MAE: 11.116439859525899


# Neural Network

In [37]:
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

In [38]:
model.fit(X_train, y_train, epochs=100, batch_size=10, validation_split=0.2)


Epoch 1/100
22/22 [==============================] - 0s 4ms/step - loss: 3153.3228 - mean_absolute_error: 55.1381 - val_loss: 3167.7593 - val_mean_absolute_error: 54.6171
Epoch 2/100
22/22 [==============================] - 0s 1ms/step - loss: 2890.5286 - mean_absolute_error: 52.6114 - val_loss: 2902.9104 - val_mean_absolute_error: 52.1122
Epoch 3/100
22/22 [==============================] - 0s 2ms/step - loss: 2548.3220 - mean_absolute_error: 49.1681 - val_loss: 2504.6392 - val_mean_absolute_error: 48.1689
Epoch 4/100
22/22 [==============================] - 0s 906us/step - loss: 2044.1639 - mean_absolute_error: 43.4431 - val_loss: 1946.2689 - val_mean_absolute_error: 42.1006
Epoch 5/100
22/22 [==============================] - 0s 680us/step - loss: 1359.7004 - mean_absolute_error: 34.4261 - val_loss: 1305.2341 - val_mean_absolute_error: 33.7476
Epoch 6/100
22/22 [==============================] - 0s 589us/step - loss: 709.7419 - mean_absolute_error: 23.2979 - val_loss: 746.5149 - val

In [39]:
loss, mae = model.evaluate(X_test, y_test)
print(f'Test MAE: {mae}')

3/3 [==============================] - 0s 353us/step - loss: 277.9709 - mean_absolute_error: 10.9090
Test MAE: 10.90898609161377
